In [1]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# nltk.download('stopwords') # ----> udkommenter denne linje, hvis du ikke har nltk installeret
# nltk.download('punkt') # ----> udkommenter denne linje, hvis du ikke har nltk installeret
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("995,000_rows.csv", dtype="string")

print("the shape of the data is: ", data.shape)

# print(data.head())

the shape of the data is:  (995000, 17)


In [88]:
# creating dictionary with domains as key and their frequency as value
author_dict = {}
i = 0

data = data.dropna(subset=['content', 'type', 'authors'])

for (author) in data['authors']:
    if author not in author_dict.keys():
        author_dict[author] = [data['type'].iloc[i]]
    else:
        # print(author + " " + data['type'][i])
        if data['type'].iloc[i] not in author_dict[author]:
            author_dict[author].append(data['type'].iloc[i])
    i = i + 1

# print(author_dict)

author_types_dict = {}
for (key, value) in author_dict.items():
    if len(value) > 1:
        author_types_dict[key] = value

sorted_dict = dict(sorted(author_types_dict.items(), key=lambda x:len(x[1]), reverse=True))

print(sorted_dict)
print(len(sorted_dict))

counter = 0

for key, value in sorted_dict.items():
    if 'fake' in value and ('reliable' in value or "political" in value):
        print(value)
        counter = counter + 1

print(counter)

# # creating plot  
# words = list(sorted_domain_dict.keys())[:50]
# freq = list(sorted_domain_dict.values())[:50]

# fig = plt.figure(figsize = (100, 5))
 
# plt.bar(words, freq, color ='green')
 
# plt.xlabel("Most frequent domains")
# plt.ylabel("Frequency")
# plt.title("50 most frequent domains in data")
# plt.show()

{'Posted On': ['unknown', 'conspiracy', 'bias', 'fake', 'junksci'], 'Michael Snyder': ['unknown', 'conspiracy', 'bias', 'political'], 'Mac Slavo': ['bias', 'unknown', 'conspiracy', 'political'], 'Mike Adams': ['junksci', 'conspiracy', 'unknown', 'bias'], 'Alex Newman': ['bias', 'conspiracy', 'fake', 'unreliable'], 'Carey Wedler': ['unreliable', 'political', 'conspiracy', 'junksci'], 'Paul Craig Roberts': ['bias', 'political', 'unknown', 'conspiracy'], 'Keith Koffler': ['clickbait', 'political', 'unknown', 'fake'], 'Amanda Froelich': ['junksci', 'bias', 'political', 'conspiracy'], 'Robert Parry': ['reliable', 'unknown', 'political', 'bias'], 'Tess Pennington': ['unknown', 'fake', 'bias', 'conspiracy'], 'John Vibes': ['unknown', 'political', 'conspiracy', 'unreliable'], 'Amanda House': ['bias', 'political', 'junksci', 'clickbait'], 'Kevin Mooney': ['bias', 'clickbait', 'conspiracy', 'political'], 'Alexander Smith': ['unreliable', 'reliable', 'fake'], 'Associated Press': ['reliable', 'pol

In [5]:
# cleaning content

def cleanText(text):
    if not isinstance(text, str):  
        return str(text)
    
    # lower case
    text = text.lower()

    # should not contain multiple spaces, tabs or newlines
    text = re.sub(r'\s+', ' ', text)

    # date and time stuff
    # text = re.sub(r'\b(?:the )?(\d{1,2})(?:st|nd|rd|th)?\s*(?:of\s+)?(January|February|March|April|May|June|July|August|September|October|November|December)\b', '<DATE>', text)
    # text = re.sub(r'r"([a-zA-Z]{3}\s\d{1,2}\s\d{4})"', '<DATE>', text, flags=re.IGNORECASE)
    # text = re.sub(r'\b(?:the )?(\d{1,2})(?:st|nd|rd|th)?(?: of)?(?: (?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?))(?: (\d{4}|\d{2}))?\b', '<DATE>', text)
    # text = re.sub(r'\b(?:the )?(\d{1,2})(?:st|nd|rd|th)?(?: of)?(?: (?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?|Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?))(?: (\d{4}|\d{2}))?\b', '<DATE>', text)

    # replace dates with <DATE>
    #  january 18, 2018. jan 18, 2018. 2018-01-18
    date_pattern = r'\b(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\s+\d{1,2}(?:,\s+|\s+)\d{4}\b|\b\d{4}-\d{2}-\d{2}\b'

    # date_pattern = r'\b(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\s+\d{1,2},\s+\d{4}\b|\b\d{4}-\d{2}-\d{2}\b'
    text = re.sub(date_pattern, '<DATE>', text)
    # nov. 5
    date_pattern2 = r'\b(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\.\s+\d{1,2}\b'
    text = re.sub(date_pattern2, '<DATE>', text)

    # text = re.sub(r'\b(?:\d{1,2}[-/th|st|nd|rd\s]*)?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)(?:\s*(?:\d{1,2}[-/th|st|nd|rd\s]*))?(?:\s*(?:\d{4}|\d{2}))?\b', '<DATE>', text)

    # replace numbers with <NUM>
    text = re.sub(r'\d+', '<NUM>', text)

    # replace urls with <URL>
    text = re.sub(r'(http|https)://[^\s]*', '<URL>', text)

    # replace emails with <EMAIL>
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w{2,4}\b', '<EMAIL>', text)

    # remove various punctuations
    text = re.sub(r'[^\w\s]', '', text)

    return text

def clean_parts(data):
    data['content'] = data['content'].apply(cleanText)
    return data

# cleaning content
data_cleaned = clean_parts(data)

In [6]:
stemmer = PorterStemmer()
stemmed_data = data_cleaned['content'].apply(lambda x:' '.join([stemmer.stem(word) for word in nltk.word_tokenize(x)]))

In [7]:
# tokenizing content
# tokens = data_cleaned['content'].apply(nltk.word_tokenize)

# removing stopwords
# stop_words = set(stopwords.words('english'))
# filtered_tokens = []

# for list in tokens:
#     filtered_words = []
#     for w in list:
#         if w not in stop_words:
#             filtered_words.append(w)
#     filtered_tokens.append(filtered_words)

stemmed_data.head(5).to_csv("995,000_stemmed.csv")

In [8]:
# vocabulary = []
# vocabulary_with_stopwords = []

# for list in tokens:
#     for w in list:
#         if w not in vocabulary_with_stopwords:
#             vocabulary_with_stopwords.append(w)


# for list in filtered_tokens:
#     for w in list:
#         if w not in vocabulary:
#             vocabulary.append(w)


# print("Vocabulary with stopwords included:\n")
# print(vocabulary_with_stopwords)
# print("Length of vocabulary with stopwords included: " + str(len(vocabulary_with_stopwords)) + "\n")
# print("Vocabulary with stopwords excluded:\n")
# print(vocabulary)
# print("Length of vocabulary with stopwords excluded: " + str(len(vocabulary)))
# print("Reduction in vocabulary length: " + str(len(vocabulary_with_stopwords) - len(vocabulary)))